# Multivariate regression Closed form

In [29]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import pandas as pd
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

main_df = pd.read_csv("insurance.csv")



#Use scikit learn
# column_to_label_encode = normalized_df["sex"]
# column_to_label_encode.head()

# label_encoder = LabelEncoder()
# label_encoded_column = label_encoder.fit_transform(column_to_label_encode)
# label_encoded_column


#Data Description

In [30]:
main_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [31]:
main_df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


# Checking for missing values

In [32]:
main_df.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

# One hot encoding

In [33]:
column_names_to_one_hot = ["sex", "smoker", "region"]

In [34]:
main_df = pd.get_dummies(main_df, columns=column_names_to_one_hot)

In [35]:
main_df.loc[:20,:]

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
5,31,25.740,0,3756.62160,1,0,1,0,0,0,1,0
6,46,33.440,1,8240.58960,1,0,1,0,0,0,1,0
7,37,27.740,3,7281.50560,1,0,1,0,0,1,0,0
8,37,29.830,2,6406.41070,0,1,1,0,1,0,0,0
9,60,25.840,0,28923.13692,1,0,1,0,0,1,0,0


In [36]:
main_df.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_female', 'sex_male',
       'smoker_no', 'smoker_yes', 'region_northeast', 'region_northwest',
       'region_southeast', 'region_southwest'],
      dtype='object')

# Checking for duplicate rows

In [37]:
main_df.index[main_df.duplicated()]
main_df.duplicated().sum()

1

In [38]:
main_df.drop(axis="rows", labels=main_df.index[main_df.duplicated()], inplace=True)

In [39]:
main_df.duplicated().sum()

0

# Normalization

In [40]:
normalized_df=(main_df-main_df.min())/(main_df.max()-main_df.min())

In [41]:
normalized_df =  (main_df-main_df.min())/(main_df.max()-main_df.min())
np.random.shuffle(normalized_df.values)

In [42]:
X = normalized_df.drop(axis="columns", labels="charges").to_numpy().astype(np.float64)
y = normalized_df["charges"].to_numpy().astype(np.float64)
ones = np.ones([X.shape[0],1])
X = np.concatenate((ones,X),axis=1)

In [43]:
len(X)

1337

In [44]:
len(y)

1337

# Train test split

In [45]:
train_num = int(1337*.8)

In [46]:
train_X,test_X = X[train_num:], X[:train_num]
train_y, test_y = y[train_num:], y[:train_num]

In [47]:
train_X


array([[1.        , 0.5       , 0.56873823, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.26086957, 0.44229217, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.26086957, 0.7306968 , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [1.        , 0.65217391, 0.34759214, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.23913043, 0.37059457, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.04347826, 0.35014797, ..., 1.        , 0.        ,
        0.        ]])

In [48]:
# train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.6)

In [49]:
train_X

array([[1.        , 0.5       , 0.56873823, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.26086957, 0.44229217, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.26086957, 0.7306968 , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [1.        , 0.65217391, 0.34759214, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.23913043, 0.37059457, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.04347826, 0.35014797, ..., 1.        , 0.        ,
        0.        ]])

In [50]:
scale = MinMaxScaler()
X_train_transformed = scale.fit_transform(train_X)
X_test_transformed = scale.transform(test_X)

In [51]:
# Build a ridge, lasso and regular linear regression model  
# Note that in scikit-learn, the regularization parameter is denoted by alpha (and not lambda)
ridge = Ridge(alpha=0.5)
ridge.fit(X_train_transformed, train_y)

lasso = Lasso(alpha=0.5)
lasso.fit(X_train_transformed, train_y)

lin = LinearRegression()
lin.fit(X_train_transformed, train_y)

LinearRegression()

In [52]:
y_h_ridge_train = ridge.predict(X_train_transformed)
y_h_ridge_test = ridge.predict(X_test_transformed)

y_h_lasso_train = lasso.predict(X_train_transformed)
y_h_lasso_test = lasso.predict(X_test_transformed)

y_h_lin_train = lin.predict(X_train_transformed)
y_h_lin_test = lin.predict(X_test_transformed)

In [53]:
print('Train Error Ridge Model', mean_squared_error(train_y, y_h_ridge_train))
print('Test Error Ridge Model', mean_squared_error(test_y, y_h_ridge_test))
print('\n')

print('Train Error Lasso Model', mean_squared_error(train_y, y_h_lasso_train))
print('Test Error Lasso Model', mean_squared_error(test_y, y_h_lasso_test))
print('\n')

print('Train Error Unpenalized Linear Model', mean_squared_error(train_y, y_h_lin_train))
print('Test Error Unpenalized Linear Model', mean_squared_error(test_y, y_h_lin_test))

Train Error Ridge Model 0.008453122857603478
Test Error Ridge Model 0.00982286300543301


Train Error Lasso Model 0.03281277548563674
Test Error Lasso Model 0.0387775400783073


Train Error Unpenalized Linear Model 0.008448586725397264
Test Error Unpenalized Linear Model 0.009806282961120991


In [54]:
print('Ridge parameter coefficients:', ridge.coef_)
print('Lasso parameter coefficients:', lasso.coef_)
print('Linear model parameter coefficients:', lin.coef_)

Ridge parameter coefficients: [ 0.          0.19277108  0.16746437  0.04631696  0.00678637 -0.00678637
 -0.18171337  0.18171337  0.02096162 -0.00572195 -0.01486898 -0.0003707 ]
Lasso parameter coefficients: [ 0.  0.  0.  0. -0.  0. -0.  0.  0. -0.  0.  0.]
Linear model parameter coefficients: [ 0.00000000e+00  1.96011659e-01  1.76382676e-01  4.70512617e-02
 -4.89453597e+09 -4.89453597e+09  2.82268731e+10  2.82268731e+10
  6.28290287e+11  6.28290287e+11  6.28290287e+11  6.28290287e+11]


In [55]:
# Prepare data
poly = PolynomialFeatures(degree=6)
X_train_poly = poly.fit_transform(train_X)
X_test_poly = poly.transform(test_X)

X_train_transformed = scale.fit_transform(X_train_poly)
X_test_transformed = scale.transform(X_test_poly)

# Fit models
ridge.fit(X_train_transformed, train_y)
lasso.fit(X_train_transformed, train_y)
lin.fit(X_train_transformed, train_y)

# Generate predictions
y_h_ridge_train = ridge.predict(X_train_transformed)
y_h_ridge_test = ridge.predict(X_test_transformed)
y_h_lasso_train = lasso.predict(X_train_transformed)
y_h_lasso_test = lasso.predict(X_test_transformed)
y_h_lin_train = lin.predict(X_train_transformed)
y_h_lin_test = lin.predict(X_test_transformed)

# Display results
print('Train Error Polynomial Ridge Model', mean_squared_error(train_y, y_h_ridge_train))
print('Test Error Polynomial Ridge Model', mean_squared_error(test_y, y_h_ridge_test))
print('\n')
print('Train Error Polynomial Lasso Model', mean_squared_error(train_y, y_h_lasso_train))
print('Test Error Polynomial Lasso Model', mean_squared_error(test_y, y_h_lasso_test))
print('\n')
print('Train Error Unpenalized Polynomial Model', mean_squared_error(train_y, y_h_lin_train))
print('Test Error Unpenalized Polynomial Model', mean_squared_error(test_y, y_h_lin_test))
print('\n')
print('Polynomial Ridge Parameter Coefficients:', len(ridge.coef_[ridge.coef_ != 0]), 
      'non-zero coefficient(s) and', len(ridge.coef_[ridge.coef_ == 0]), 'zeroed-out coefficient(s)')
print('Polynomial Lasso Parameter Coefficients:',  len(lasso.coef_[lasso.coef_ != 0]), 
      'non-zero coefficient(s) and', len(lasso.coef_[lasso.coef_ == 0]), 'zeroed-out coefficient(s)')
print('Polynomial Model Parameter Coefficients:',  len(lin.coef_[lin.coef_ != 0]), 
      'non-zero coefficient(s) and', len(lin.coef_[lin.coef_ == 0]), 'zeroed-out coefficient(s)')

Train Error Polynomial Ridge Model 0.0016138358485384985
Test Error Polynomial Ridge Model 0.01687987127390862


Train Error Polynomial Lasso Model 0.03281277548563674
Test Error Polynomial Lasso Model 0.0387775400783073


Train Error Unpenalized Polynomial Model 9.939502088516175e-29
Test Error Unpenalized Polynomial Model 3.4198740616165852


Polynomial Ridge Parameter Coefficients: 8339 non-zero coefficient(s) and 10225 zeroed-out coefficient(s)
Polynomial Lasso Parameter Coefficients: 0 non-zero coefficient(s) and 18564 zeroed-out coefficient(s)
Polynomial Model Parameter Coefficients: 8368 non-zero coefficient(s) and 10196 zeroed-out coefficient(s)
